In [1]:
import pandas as pd
import numpy as np
import hdf5_getters as hd
import glob
import sklearn
from sklearn import preprocessing

In [2]:
#To see if all the 10,000 files are available for data extraction
h5_dir = "MillionSongSubset" #<file_path where the song_dataset is stored>

# Get a list of all .h5 files in the directory and its subdirectories
h5_files = glob.glob(h5_dir + '/**/*.h5', recursive=True)
len(h5_files) #should be 10,000

10000

In [3]:
#Reading the dataset
df = pd.read_csv("3593_song_newfeatures_genre_dataset.csv")

In [4]:
#Filtering out data for Folk genre 
df_song = df[df['genre'] == 'Folk']
df_song

,Unnamed: 0,id,spectral_centroid_mean,spectral_centroid_dev,rolloff_point_mean,rolloff_point_dev,flux_mean,flux_dev,variability_mean,variability_dev,compactness_mean,compactness_dev,rms_mean,rms_dev,low_energy_window_mean,low_energy_window_dev,zero_crossing_mean,zero_crossing_dev,title,genre
20,20,TRBGRKJ128F4261513,4.410,5.880,0.02648,0.05659,0.000334,0.000858,0.001396,0.001347,1658.0,209.1,0.05356,0.04899,0.6085,0.07217,13.500,14.090,b'Too Far From The Tree',Folk
95,95,TRBGYDZ12903D075D4,4.722,3.045,0.02898,0.02600,0.001745,0.003132,0.004503,0.002648,1820.0,308.6,0.16740,0.09062,0.5711,0.06888,12.840,9.215,b'All Creatures Of Our God And King',Folk
292,292,TRBBMQO128F425559F,6.470,3.956,0.04676,0.03385,0.000277,0.000269,0.001808,0.000790,1809.0,258.5,0.06770,0.02707,0.5435,0.03777,20.610,10.680,b'Red Apple Juice (LP Version)',Folk
298,298,TRBBGDK128F427D450,6.077,10.170,0.04085,0.07501,0.001695,0.002407,0.004369,0.002500,1642.0,215.6,0.16140,0.08794,0.5360,0.05588,18.410,24.430,b'My Maria (LP Version)',Folk
471,471,TRBFDHN128F4242963,6.134,2.936,0.03964,0.02487,0.001112,0.001145,0.003512,0.001422,1722.0,233.9,0.13490,0.05144,0.5417,0.05209,17.240,7.890,b'Ella Speed',Folk
569,569,TRBAPBU128F9304E1B,6.794,3.071,0.03338,0.03223,0.000476,0.000791,0.002538,0.001080,1636.0,159.2,0.09737,0.03807,0.5505,0.05484,18.910,9.278,b'The Carrying Arms',Folk
582,582,TRBAUAB128F92FB27F,5.129,3.758,0.03529,0.03169,0.001585,0.002338,0.003794,0.001580,1801.0,287.3,0.14000,0.05480,0.5562,0.06819,16.330,10.700,b'Outubro',Folk
619,619,TRBAKER128F1495617,15.190,5.851,0.09339,0.04598,0.000850,0.000820,0.003004,0.001021,1818.0,183.8,0.12030,0.03877,0.5216,0.04598,42.050,13.860,b'We Three Kings',Folk
655,655,TRBEIBI12903CC4C7B,9.092,4.308,0.05459,0.03550,0.000418,0.000351,0.002294,0.001017,1668.0,166.6,0.08853,0.03738,0.5383,0.05180,27.980,10.420,b'Losing True (Album Version)',Folk
824,824,TRAMCHM128EF365672,10.210,10.550,0.05905,0.06113,0.000100,0.000221,0.000877,0.000874,1644.0,252.6,0.03395,0.03319,0.5595,0.09286,24.100,22.120,b'Shaggy Dog Songs',Folk


In [5]:
#Creating lists of Genre and Track_id
track_id = df_song['id'].tolist()
genre = df_song['genre'].tolist()

In [6]:
genre

['Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk',
 'Folk']

In [7]:
#Function to extract the cleaned track_id
def clean(text):
    text = str(text)
    text = text.replace("b'", "")
    text = text[:-1]
    return text

In [8]:
#Collecting segment info for Loudness only for those songs in the 3953_dataset
batch_size = 10

song_id = []
loud_max_seg = []
pitch_seg = []
timbre_seg = []

for i in range(0, len(h5_files), batch_size):
    # Get the batch of files
    batch_files = h5_files[i:i+batch_size]
    
    # Loop through the files in the batch
    for file_name in batch_files:
        h5 = hd.open_h5_file_read(file_name)
        
        id = hd.get_track_id(h5)
        id = clean(id)

        if id in track_id:
            #print("Collecting data")
            song_id.append(hd.get_track_id(h5))
            loud_max_seg.append(hd.get_segments_loudness_max(h5))
            timbre_seg.append(hd.get_segments_timbre(h5))
            pitch_seg.append(hd.get_segments_pitches(h5))
        
        h5.close()

In [9]:
#All are equal to the number of songs filtered out from the Folk genre
len(song_id), len(loud_max_seg), len(timbre_seg), len(pitch_seg), len(genre)

(48, 48, 48, 48, 48)

In [10]:
#Inspecting values of loudness for different segments for 2 songs.
loud_max_seg[0:2]

[array([-58.582, -20.229, -18.838, -19.031, -18.154, -20.752, -25.576,
        -21.905, -23.619, -24.393, -25.963, -26.066, -30.087, -30.655,
        -30.261, -27.21 , -27.796, -27.437, -32.063, -25.727, -29.325,
        -26.43 , -33.644, -26.09 , -28.147, -24.758, -31.688, -24.566,
        -23.136, -27.634, -28.518, -12.7  , -17.753, -10.589, -18.842,
        -17.835, -21.813, -23.693, -19.506, -25.057, -30.544, -31.243,
        -36.134, -26.938, -27.194, -21.997, -27.216, -23.159, -16.843,
        -16.389, -18.972, -19.311, -29.641, -23.783, -27.5  , -28.492,
        -30.615, -22.96 , -21.462, -22.051, -23.937, -22.889, -11.789,
        -10.892, -12.996, -17.988, -12.44 , -11.282, -14.708, -12.893,
        -18.62 , -15.183, -17.675, -26.479, -25.641, -23.76 , -32.464,
        -25.194, -25.574, -14.654, -20.113, -21.3  , -23.816, -23.012,
        -26.877, -30.287, -27.579, -28.152, -24.476, -23.444, -24.692,
        -27.654, -24.648, -19.83 , -11.428, -13.317, -10.735, -14.83 ,
      

In [11]:
#inspecting the segments' shape for all songs
for arr in loud_max_seg:
    print(arr.shape)

(532,)
(783,)
(700,)
(1297,)
(725,)
(406,)
(631,)
(863,)
(606,)
(84,)
(717,)
(921,)
(765,)
(843,)
(531,)
(582,)
(586,)
(585,)
(596,)
(940,)
(329,)
(856,)
(761,)
(769,)
(692,)
(643,)
(478,)
(515,)
(841,)
(658,)
(811,)
(1018,)
(1383,)
(554,)
(546,)
(827,)
(741,)
(677,)
(513,)
(1004,)
(467,)
(946,)
(616,)
(451,)
(796,)
(626,)
(528,)
(480,)


In [12]:
#Installing Librosa Library for creating spectrogram
!pip install librosa
import librosa
import librosa.display
import numpy as np

In [13]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# Padding the loudness arrays to be of the same length 
max_length = max(len(arr) for arr in loud_max_seg)
padded_loud_max_seg = []
for arr in loud_max_seg:
    padded_arr = np.pad(arr, (0, max_length - len(arr)), mode='constant')
    padded_loud_max_seg.append(padded_arr)
concatenated_loudness = np.vstack(padded_loud_max_seg)

# Standardization (Z-score normalization)
mean_val = np.mean(concatenated_loudness)
std_val = np.std(concatenated_loudness)
scaled_loudness = (concatenated_loudness - mean_val) / std_val

sr= 22050

# Computing the spectrogram using librosa
spec = librosa.feature.melspectrogram(y=None, sr=sr, S=scaled_loudness, n_fft=2048, hop_length=512, n_mels=64)

# Converting the power spectrogram to dB scale
spec_db = librosa.power_to_db(spec, ref=np.max)

# Plotting the spectrogram
plt.figure(figsize=(10, 5))
librosa.display.specshow(spec_db, x_axis='time', y_axis='mel', sr=sr, hop_length=512, cmap='coolwarm')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram for Folk genre')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.show()

C:\Users\anush\anaconda3\lib\site-packages\librosa\feature\spectral.py:2157: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
